# 무신사 스토어 홈페이지 Bestseller 전체페이지(100page) 가져오기


In [4]:
import requests
from bs4 import BeautifulSoup

# pymysql을 이용해 MySQL 로그인
import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='watson1259@', db='musinsa', charset='utf8')
cursor = db.cursor()


# 여러 페이지 다중 Requests
url = "https://store.musinsa.com/app/contents/bestranking/?d_cat_cd=&u_cat_cd=&range=nw&price=&ex_soldout=&sale_goods=&new_product_yn=&list_kind="

# 무신사 홈페이지 특성상 페이지가 넘어갈 때 마다 url 끝 주소가 'page=페이지번호'로 바뀜에 주목하여 간단하게 다중 페이지 요청 가능
for num in range(100):
    print(f"Scrapping Page: {num+1}")
    res = requests.get(f"{url}&page={num+1}")
    get_items(res)
db.commit()
db.close()

Scrapping Page: 1
Scrapping Page: 2
Scrapping Page: 3
Scrapping Page: 4
Scrapping Page: 5
Scrapping Page: 6
Scrapping Page: 7
Scrapping Page: 8
Scrapping Page: 9
Scrapping Page: 10
Scrapping Page: 11
Scrapping Page: 12
Scrapping Page: 13
Scrapping Page: 14
Scrapping Page: 15
Scrapping Page: 16
Scrapping Page: 17
Scrapping Page: 18
Scrapping Page: 19
Scrapping Page: 20
Scrapping Page: 21
Scrapping Page: 22
Scrapping Page: 23
Scrapping Page: 24
Scrapping Page: 25
Scrapping Page: 26
Scrapping Page: 27
Scrapping Page: 28
Scrapping Page: 29
Scrapping Page: 30
Scrapping Page: 31
Scrapping Page: 32
Scrapping Page: 33
Scrapping Page: 34
Scrapping Page: 35
Scrapping Page: 36
Scrapping Page: 37
Scrapping Page: 38
Scrapping Page: 39
Scrapping Page: 40
Scrapping Page: 41
Scrapping Page: 42
Scrapping Page: 43
Scrapping Page: 44
Scrapping Page: 45
Scrapping Page: 46
Scrapping Page: 47
Scrapping Page: 48
Scrapping Page: 49
Scrapping Page: 50
Scrapping Page: 51
Scrapping Page: 52
Scrapping Page: 53
Sc

In [3]:
# 제품정보(순위, 브랜드, 제품명, 가격(할인가격 적용)) 크롤링

def get_items(html):
    soup = BeautifulSoup(html.content, 'html.parser')
    products = soup.select("div.list-box.box li")
    for product in products:
        data_dict = dict()
        result = product.select_one("div.article_info")
        if result != None:
            if result.select_one('p.item_title:nth-of-type(0) > span'):
                brand = result.select_one('p.item_title:nth-of-type(1)').get_text()
            else:
                brand = result.select_one('p.item_title > a').get_text()
            title = result.select_one('p.list_info > a').get_text().strip()
            price = result.select_one('p.price')
            origin = price.select_one('del')
            if origin:
                _ = origin.extract()
                price = price.get_text().strip().replace('원','').replace(',','')
            else:
                price = price.get_text().strip().replace('원','').replace(',','')    

        ranking = product.find('p', {'class':'n-label'})
        if ranking != None:
            increase = ranking.find('span')
            if increase:
                _ = increase.extract()
                ranking = ranking.get_text().strip().replace('위','')
            else:
                ranking = ranking.get_text().strip().reaplace('위','')

            #print(ranking, brand, title, price)
            
            data_dict['ranking'] = ranking
            data_dict['brand'] = brand
            data_dict['title'] = title
            data_dict['price'] = price
            save_data(data_dict)
            
            

In [2]:
def save_data(items_info):
    sql = """INSERT INTO musinsa_best_products values("""+str(items_info['ranking'])+""",'"""+items_info['brand']+"""',
    '"""+items_info['title']+"""',"""+str(items_info['price'])+""");"""
    cursor.execute(sql)

# 크롤링 데이터 집어넣을 DB 만들기

In [1]:
import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='watson1259@', db='musinsa', charset='utf8')
cursor = db.cursor()

sql = '''
    CREATE TABLE musinsa_best_products (
        Ranking int not null primary key,
        Brand varchar(30) not null,
        Product_title varchar(100) not null,
        Price int not null
);
'''
cursor.execute(sql)
db.commit()
db.close()

# 직접 넣은 데이터 SQL 쿼리를 통해 분석

In [5]:
# 베스트 제품에 속하는 브랜드와 브랜드별 제품 갯수 출력해보고 제일 많은 상위 브랜드가 뭘까?

import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='watson1259@', db='musinsa', charset='utf8')
cursor = db.cursor()

sql = '''SELECT Brand, COUNT(*) FROM musinsa_best_products GROUP BY Brand ORDER BY COUNT(*) DESC;'''
cursor.execute(sql)
results = cursor.fetchall()
for result in results:
    print(result)

('무신사 스탠다드', 450)
('커버낫', 262)
('아디다스', 184)
('디스이즈네버댓', 166)
('나이키', 154)
('마크 곤잘레스', 117)
('엘엠씨', 103)
('유니폼브릿지', 95)
('로맨틱크라운', 90)
('라퍼지스토어', 89)
('더블유브이프로젝트', 87)
('반스', 82)
('휠라', 82)
('칼하트', 82)
('드로우핏', 81)
('앤더슨벨', 76)
('꼼파뇨', 73)
('스컬프터', 71)
('키르시', 70)
('커스텀어클락', 63)
('엘무드', 61)
('네스티팬시클럽', 59)
('누깍', 58)
('파르티멘토', 57)
('비바스튜디오', 55)
('디스커버리 익스페디션', 54)
('5252 바이 오아이오아이', 54)
('마하그리드', 54)
('프리즘웍스', 53)
('포텐스타', 51)
('제멋', 49)
('에즈카톤', 49)
('미나브', 49)
('오드원아웃', 49)
('챔피온', 48)
('엠엠엘지', 48)
('내셔널지오그래픽', 47)
('캉골', 45)
('쿠어', 43)
('리복', 43)
('트래블', 42)
('필루미네이트', 41)
('레이디 볼륨', 41)
('제로', 39)
('소버먼트', 39)
('모디파이드', 36)
('86로드', 36)
('라이풀', 36)
('로맨틱 파이어리츠', 34)
('컨버스', 34)
('티떠블유엔', 32)
('피스워커', 31)
('멜란지 마스터', 30)
('뉴발란스', 30)
('니티드', 30)
('세인트페인', 30)
('미뇽네프', 30)
('로사케이', 30)
('브랜디드', 29)
('푸마', 29)
('메인부스', 29)
('베테제', 29)
('에드', 29)
('앨리스마샤', 29)
('에스피오나지', 28)
('블론드나인', 28)
('그루브라임', 27)
('메종미네드', 27)
('엠엘비', 27)
('키르시 X 로맨틱크라운', 27)
('다이아몬드 레이라', 26)
('크럼프', 26)
('후아유'

### → 전체 Bestseller 중에서 가장 많은 점유율은 차지하는 브랜드는 ' 무신사 스탠다드 ' 브랜드이며 다음 순위로는 '커버낫', '아디다스' , '디스이즈네버댓', '나이키' 이다.

### → 최근 무신사 자체에서 제작한 브랜드인 '무신사 스탠다드'가 큰 인기를 끌고 있다. 해당 브랜드는 다른 브랜드 처럼 비싼 가격의 제품이 아닌 이른바 '가성비' 가격에 비해 성능이 좋은 제품들을 연이어 출시하고 있다.

In [7]:
# Bsetseller들의 제품들 평균 가격은 얼마일까?

import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='watson1259@', db='musinsa', charset='utf8')
cursor = db.cursor()

sql = """SELECT AVG(price) FROM musinsa_best_products;"""
cursor.execute(sql)
result = cursor.fetchone()
print(result[0])


65512.4404


### →  Bestseller 제품들의 평균 가격은 약 65,000원 대이다. 

In [20]:
# 베스트제품이 가장 많은 브랜드인 무신사스탠다드의 평균 제품가격을 알아보자.

import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='watson1259@', db='musinsa', charset='utf8')
cursor = db.cursor()

sql = """SELECT Brand, AVG(price) FROM musinsa_best_products WHERE Brand = '무신사 스탠다드';"""
cursor.execute(sql)
result = cursor.fetchone()
print(result)       # Decimal 데이터 형식은 숫자에 대 한 최대 유효 자릿수를 제공
print()
print()
print()
print()

# 베스트제품 순위 안에 드는 제품이 50개 이상인 브랜드별로 평균가격을 비교해보자

sql = """SELECT Brand, AVG(price), COUNT(*) FROM musinsa_best_products GROUP BY Brand HAVING COUNT(*) > 50 ORDER BY AVG(price);"""
cursor.execute(sql)
results = cursor.fetchall()
for index, result in enumerate(results):
    print(index+1, f"- 브랜드:{result[0]} / 가격:{result[1]}원 / 제품개수:{result[2]}개")


('무신사 스탠다드', Decimal('35360.3333'))




1 - 브랜드:무신사 스탠다드 / 가격:35360.3333원 / 제품개수:450개
2 - 브랜드:네스티팬시클럽 / 가격:36924.5763원 / 제품개수:59개
3 - 브랜드:꼼파뇨 / 가격:40494.5205원 / 제품개수:73개
4 - 브랜드:휠라 / 가격:40853.6585원 / 제품개수:82개
5 - 브랜드:마하그리드 / 가격:41866.6667원 / 제품개수:54개
6 - 브랜드:커스텀어클락 / 가격:42783.3333원 / 제품개수:63개
7 - 브랜드:더블유브이프로젝트 / 가격:44828.7356원 / 제품개수:87개
8 - 브랜드:스컬프터 / 가격:49971.8310원 / 제품개수:71개
9 - 브랜드:파르티멘토 / 가격:51268.4211원 / 제품개수:57개
10 - 브랜드:키르시 / 가격:52108.5714원 / 제품개수:70개
11 - 브랜드:엘엠씨 / 가격:52869.9029원 / 제품개수:103개
12 - 브랜드:라퍼지스토어 / 가격:56670.7865원 / 제품개수:89개
13 - 브랜드:아디다스 / 가격:59000.5435원 / 제품개수:184개
14 - 브랜드:5252 바이 오아이오아이 / 가격:60412.0370원 / 제품개수:54개
15 - 브랜드:칼하트 / 가격:61063.2927원 / 제품개수:82개
16 - 브랜드:마크 곤잘레스 / 가격:61387.8632원 / 제품개수:117개
17 - 브랜드:나이키 / 가격:62812.9870원 / 제품개수:154개
18 - 브랜드:커버낫 / 가격:64139.8855원 / 제품개수:262개
19 - 브랜드:로맨틱크라운 / 가격:66988.8889원 / 제품개수:90개
20 - 브랜드:반스 / 가격:69700.0000원 / 제품개수:82개
21 - 브랜드:유니폼브릿지 / 가격:75888.4211원 / 제품개수:95개
22 - 브랜드:디스이즈네버댓 / 가격:76300.0000원 / 제품개수:166개
23 - 브랜드:프

### → 평균값의 오류를 막기 위해 브랜드 별 제품 갯수가 50개 이상 베스트제품에 들어가는 브랜드만 추출하였다.

### → 50개 이상만 했는데도 '무신사 스탠다드' 브랜드가 450개라는 엄청난 많은 숫자의 갯수임에도 불구하고 평균가격이 가장 낮음을 기록했다.

### → 이에 따라 '무신사 스탠다드' 제품이 이른바 '가성비' 라는 제품임을 방증할 수 있다.

In [25]:
# 가장 비싼 제품과 가장 싼 제품의 각 브랜드, 제품명, 가격을 알아보자.

import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='watson1259@', db='musinsa', charset='utf8')
cursor = db.cursor()

sql = """SELECT Brand, Product_title, MAX(price) FROM musinsa_best_products;"""
cursor.execute(sql)
result = cursor.fetchone()
print(f"1.가장 비싼 브랜드: {result[0]} / 제품명: {result[1]} / 가격: {result[2]}")
print()
print()


sql = """SELECT Brand, Product_title, MIN(price) FROM musinsa_best_products;"""
cursor.execute(sql)
result = cursor.fetchone()
print(f"2.가장 싼 브랜드: {result[0]} / 제품명: {result[1]} / 가격: {result[2]}")


1.가장 비싼 브랜드: 무신사 스탠다드 / 제품명: 베이식 블레이저 [블랙] / 가격: 775000


2.가장 싼 브랜드: 무신사 스탠다드 / 제품명: 베이식 블레이저 [블랙] / 가격: 3000


### → 가장 비싼 브랜드와 싼 브랜드가 모두 똑같은 제품이지만 가격이 매우 상이하다. 이는 무신사 스토어 홈페이지 측에서 실수가 있는 것 같다. 그렇다면 모든 제품을 가격과 제품갯수 별로 내림차순, 올림차순으로 각각 출력해보자

In [27]:
# 모든 제품, 가격, 제품갯수 내림차순으로 출력해서 가장 비싼 제품이 무엇인지 알아보자.

import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='watson1259@', db='musinsa', charset='utf8')
cursor = db.cursor()

sql = """SELECT Brand, Product_title, price, COUNT(*) FROM musinsa_best_products GROUP BY Product_title ORDER BY price DESC;"""
cursor.execute(sql)
results = cursor.fetchall()
for index, result in enumerate(results):
    print(index+1, f"브랜드:{result[0]} / 제품명:{result[1]} / 가격:{result[2]} / 제품개수:{result[3]}")    

1 브랜드:몽클레어 / 제품명:테렌스 1038900 01AJQ 002 남자 스니커즈 / 가격:775000 / 제품개수:1
2 브랜드:오프 화이트 / 제품명:후드집업 OMBE001R19003012 0288 WHITE /남성 자켓 / 가격:719000 / 제품개수:1
3 브랜드:앤더슨벨 / 제품명:콘트라스트 스티치 레더 지퍼 트러커 자켓  awa251m(BLACK) / 가격:690000 / 제품개수:1
4 브랜드:오프 화이트 / 제품명:후드티 OMBB034R19003015 1028 BLACK/후드 긴팔티셔츠 / 가격:683000 / 제품개수:1
5 브랜드:스톤아일랜드 / 제품명:와펜 라이트 소프트쉘 721540927 V0020 남자 자켓 / 가격:676000 / 제품개수:1
6 브랜드:톰브라운 / 제품명:사선 완장 MJT021H 00535 068 남자 긴팔 맨투맨 티셔츠 / 가격:556000 / 제품개수:1
7 브랜드:아이더 / 제품명:제롬 여성 자켓 / 가격:530000 / 제품개수:1
8 브랜드:골든구스 / 제품명:19FW  메탈로고 실버탭 슈퍼스타 우먼 W77 / 가격:525000 / 제품개수:1
9 브랜드:앤더슨벨 / 제품명:조셉 베이지 울 체크 롱 코트 awa248m(BEIGE) / 가격:465000 / 제품개수:1
10 브랜드:리버클래시 / 제품명:네이비 더블라펠 세미오버핏 트렌치코트 LFS20207 / 가격:450000 / 제품개수:1
11 브랜드:베이프 / 제품명:AAPE WARRIORS HOOD / 가격:448000 / 제품개수:1
12 브랜드:리버클래시 / 제품명:2019 SS 트렌치코트 LFS20211 STANDARDFIT / 가격:427000 / 제품개수:1
13 브랜드:에디션 / 제품명:베지터블 돌체 워싱 가죽 재킷_NEY3LJ1902 / 가격:399900 / 제품개수:1
14 브랜드:밀레 클래식 / 제품명:X 크리틱 고어텍스 피쉬 테일 자켓 라이트 카키 / 가격:399000 / 제품개수:1
15 브랜드:로사케이 / 제품명:마이리본 미니숄더

633 브랜드:유니폼브릿지 / 제품명:20ss denim trucker jacket indigo / 가격:139000 / 제품개수:1
634 브랜드:유니폼브릿지 / 제품명:oversized pocket top black / 가격:139000 / 제품개수:1
635 브랜드:유니폼브릿지 / 제품명:20ss denim trucker jacket black / 가격:139000 / 제품개수:1
636 브랜드:유니폼브릿지 / 제품명:3/30 배송 oversized pocket top check / 가격:139000 / 제품개수:1
637 브랜드:커버낫 / 제품명:투 버튼 오버핏 블레이저(에코-프렌들리) 블랙 / 가격:139000 / 제품개수:1
638 브랜드:디스커버리 익스페디션 / 제품명:버킷 디원더 (WHITE) / 가격:139000 / 제품개수:1
639 브랜드:칼하트 / 제품명:덕 액티브 퀼트 자켓 블랙 / J140-001 / 가격:139000 / 제품개수:1
640 브랜드:디스커버리 익스페디션 / 제품명:버킷 디워커 V2 (Mirage Brown) / 가격:139000 / 제품개수:1
641 브랜드:디스커버리 익스페디션 / 제품명:버킷 디워커 V2 (O/WHITE) / 가격:139000 / 제품개수:1
642 브랜드:디스커버리 익스페디션 / 제품명:버킷 디워커 V2 (WHITE) / 가격:139000 / 제품개수:1
643 브랜드:디스커버리 익스페디션 / 제품명:버킷 디워커 V2 (BLACK) / 가격:139000 / 제품개수:1
644 브랜드:디스이즈네버댓 / 제품명:CORDURA® 330D Nylon SP Backpack Black / 가격:139000 / 제품개수:1
645 브랜드:나이키 / 제품명:우먼스 에어포스1 세이지 로우 (AR5339-100) / 가격:139000 / 제품개수:1
646 브랜드:디스커버리 익스페디션 / 제품명:버킷 디원더 (MULTI) / 가격:139000 / 제품개수:1
647 브랜드:디스이즈네버댓 / 제품명:Work Jacke

1133 브랜드:뉴발란스 / 제품명:NBPDAS451W/ MR530SI / 가격:99000 / 제품개수:1
1134 브랜드:뉴발란스 / 제품명:NBPDAS165B / MR530SD / 가격:99000 / 제품개수:1
1135 브랜드:나이키 / 제품명:아카데미 바람막이 자켓 블랙 / 가격:99000 / 제품개수:1
1136 브랜드:모드나인 / 제품명:After Glow - MOD1 crop / 가격:99000 / 제품개수:1
1137 브랜드:뉴발란스 / 제품명:[NBPT9B401E] 뉴발란스 608-(SMU)베이지 / 가격:99000 / 제품개수:1
1138 브랜드:디스이즈네버댓 / 제품명:Zip Cargo Pant Sage / 가격:99000 / 제품개수:1
1139 브랜드:디스이즈네버댓 / 제품명:Track Jacket Dark Green / 가격:99000 / 제품개수:1
1140 브랜드:디스이즈네버댓 / 제품명:Track Jacket Navy / 가격:99000 / 제품개수:1
1141 브랜드:아식스 / 제품명:젤 1090-W - 화이트 / 1022A215-101 / 가격:99000 / 제품개수:1
1142 브랜드:아식스 / 제품명:젤 1090 SL - 화이트 / 1023A063-100 / 가격:99000 / 제품개수:1
1143 브랜드:유니폼브릿지 / 제품명:hooded shirts khaki / 가격:99000 / 제품개수:1
1144 브랜드:유니폼브릿지 / 제품명:relax cotton jacket natural / 가격:99000 / 제품개수:1
1145 브랜드:유니폼브릿지 / 제품명:relax cotton jacket black / 가격:99000 / 제품개수:1
1146 브랜드:스테레오 바이널즈 / 제품명:3/9 배송 [SS20 SV X Sesame Street] 3D Printed Hoodie(Black) / 가격:99000 / 제품개수:1
1147 브랜드:스테레오 바이널즈 / 제품명:3/9 배송 [SS20 SV X Sesame Street]

1632 브랜드:푸마 / 제품명:푸마 X RHUDE 티 - 블랙 / 59675801 / 가격:84000 / 제품개수:1
1633 브랜드:에스피오나지 / 제품명:Kent M92 Pullover Shirts Dust / 가격:84000 / 제품개수:1
1634 브랜드:에스피오나지 / 제품명:Kent M92 Pullover Shirts Off White / 가격:84000 / 제품개수:1
1635 브랜드:칼하트 / 제품명:레가시 D89 메신저백-BLACK / 가격:84000 / 제품개수:1
1636 브랜드:널디 / 제품명:NY 트랙 탑 화이트 / 가격:84000 / 제품개수:1
1637 브랜드:에이지 / 제품명:에이지 탑 스니커즈 AGE TOP SNEAKERS (AGFT-CR-TOP-W/R013) / 가격:84000 / 제품개수:1
1638 브랜드:아디다스 / 제품명:M ID WUPO 스웻셔츠 - 블랙 / FN1663 / 가격:83900 / 제품개수:1
1639 브랜드:아디다스 / 제품명:머스트해브 스커트 - 네이비 / GL4243 / 가격:83900 / 제품개수:1
1640 브랜드:세븐셀라 / 제품명:사이드 셔링 조거 슬랙스 / 가격:83700 / 제품개수:1
1641 브랜드:쟈니웨스트 / 제품명:Ponte Single Coat LD / 가격:83600 / 제품개수:1
1642 브랜드:널디 / 제품명:코치 자켓 네이비 / 가격:83400 / 제품개수:1
1643 브랜드:리베르텡 / 제품명:헤비 양털 자켓 크림 / 가격:83400 / 제품개수:1
1644 브랜드:코모도 / 제품명:[FORMAL] 멜란지 트윌 밀드 수트 팬츠 1906192973114 / 가격:83400 / 제품개수:1
1645 브랜드:크럼프 / 제품명:벨벳 리버시블 숏패딩 (CO0032) / 가격:83300 / 제품개수:1
1646 브랜드:유니폼브릿지 / 제품명:19fw civil pullover anorak jacket black / 가격:83300 / 제품개수:1
1647 브랜드:커버낫 / 제품명

2132 브랜드:에잇세컨즈 / 제품명:베이지 솔리드 싱글 트렌치 코트 (11923TAY2A) / 가격:71900 / 제품개수:1
2133 브랜드:반스 / 제품명:빅 V 로고 크루 플리스 II - 화이트 / VN0A3TXHWHT1 / 가격:71900 / 제품개수:1
2134 브랜드:반스 / 제품명:빅 V 로고 크루 플리스 II - 블랙 / VN0A3TXHBLK1 / 가격:71900 / 제품개수:1
2135 브랜드:마이애미프로젝트 / 제품명:1791 스톡턴 천연소가죽 독일군 스니커즈 제트화이트 / 가격:71800 / 제품개수:1
2136 브랜드:휴먼갤러리 / 제품명:롱 트렌치코트_베이지 / 가격:71600 / 제품개수:1
2137 브랜드:유니폼브릿지 / 제품명:19fw civil pullover anorak jacket khaki / 가격:71400 / 제품개수:1
2138 브랜드:널디 / 제품명:베이식 로고 아노락 자켓 블랙 / 가격:71400 / 제품개수:1
2139 브랜드:로라로라 / 제품명:(OP-20316) FLOWER PATTERN COLLAR ONE-PIECE LIGHT BLUE / 가격:71250 / 제품개수:1
2140 브랜드:페이탈리즘 / 제품명:#0199 Sensitive -B standard fit / 가격:71200 / 제품개수:1
2141 브랜드:피스워커 / 제품명:Ash Worker / New Crop / 가격:71200 / 제품개수:1
2142 브랜드:랩101 / 제품명:TOM BASIC WASH CARPENTER DENIM / 가격:71200 / 제품개수:1
2143 브랜드:스컬프터 / 제품명:Double Pouch Nylon Backpack [BLACK] / 가격:71200 / 제품개수:1
2144 브랜드:페이탈리즘 / 제품명:#0215 Melt black straight fit / 가격:71200 / 제품개수:1
2145 브랜드:모디파이드 / 제품명:M1816 에센셜 세미오버핏 워시드 트러커 자켓 (블랙) / 가격:71200 / 

...

5381 브랜드:제멋 / 제품명:[기획특가] 유스 와이드 카고팬츠 4종 카키 YHLP2159 / 가격:37900 / 제품개수:1
5382 브랜드:오앤에이 / 제품명:2ND 레드플래닛 후드[블랙] / 가격:37900 / 제품개수:1
5383 브랜드:엔더즌 / 제품명:[2차 재입고] 보아 프리스 우븐 집업 [Ivory] / 가격:37900 / 제품개수:1
5384 브랜드:오앤에이 / 제품명:2ND 레드플래닛 후드[퍼플] / 가격:37900 / 제품개수:1
5385 브랜드:오앤에이 / 제품명:2ND 레드플래닛 후드[그레이] / 가격:37900 / 제품개수:1
5386 브랜드:후러브스아트 / 제품명:유니섹스 세미 와이드 크롭 밴딩 슬랙스 / 가격:37900 / 제품개수:1
5387 브랜드:챔피온 / 제품명:(2PACK) S700 무지 후드&1301 무지 반팔티 / 가격:37900 / 제품개수:1
5388 브랜드:엔더즌 / 제품명:[1차 재입고] 보아 프리스 우븐 집업 [Black] / 가격:37900 / 제품개수:1
5389 브랜드:휠라 / 제품명:세리프 헤리티지 자수 후디 - 베이지 / FE2POB6112XBEG / 가격:37900 / 제품개수:1
5390 브랜드:아디다스 / 제품명:머스트해브 박스 티 - 블랙 / FL3991 / 가격:37900 / 제품개수:1
5391 브랜드:엠블러 / 제품명:Ski bear 오버핏 후드티셔츠  AHP613(navy) / 가격:37900 / 제품개수:1
5392 브랜드:티떠블유엔 / 제품명:소프트레터 후드 회색 STHD3170 / 가격:37900 / 제품개수:1
5393 브랜드:티떠블유엔 / 제품명:소프트레터 후드 아이보리 STHD3170 / 가격:37900 / 제품개수:1
5394 브랜드:멜란지 마스터 / 제품명:스탠다드 후드 스웨트 집업 - 네이비 / 가격:37900 / 제품개수:1
5395 브랜드:멜란지 마스터 / 제품명:스탠다드 후드 스웨트 집업  - 차콜 그레이 / 가격:37900 / 제품개수:1
5396 브랜드:더블유브

5880 브랜드:나이키 / 제품명:테크 스몰 아이템백 BA5268_021 / 가격:32800 / 제품개수:1
5881 브랜드:나이키 / 제품명:여성용 클럽 로고2 레깅스 815997_010 / 가격:32800 / 제품개수:1
5882 브랜드:미니멀프로젝트 / 제품명:피그먼트 워시드 스몰 폰트 긴팔티셔츠 MLT107 [DARKGRAY] / 가격:32800 / 제품개수:1
5883 브랜드:미니멀프로젝트 / 제품명:피그먼트 워시드 스몰 폰트 긴팔티셔츠 MLT107 [WHITE] / 가격:32800 / 제품개수:1
5884 브랜드:이에스엔 / 제품명:[세트]  하프 폴라 니트 티셔츠 / 가격:32760 / 제품개수:1
5885 브랜드:크리틱 / 제품명:TRANSFORM HOODIE(M/GRAY)_CTONPHD02UC4 / 가격:32700 / 제품개수:1
5886 브랜드:씨엘론다 / 제품명:블랙 뿔테 안경 BANK BLACK 뱅크 블랙 / 가격:32700 / 제품개수:1
5887 브랜드:후러브스아트 / 제품명:3/17 배송 유니섹스 스트레이트 크롭 밴딩 슬랙스 / 가격:32600 / 제품개수:1
5888 브랜드:나이키 / 제품명:레거시 91 테크커스텀 드라이핏 볼캡 블랙 (892652-010) / 가격:32500 / 제품개수:1
5889 브랜드:꼼파뇨 / 제품명:스테인드 글라스 맵스 후드 세레니티 / 가격:32400 / 제품개수:1
5890 브랜드:트릴리온 / 제품명:[교통카드중복방지] 소가죽 사피아노 카드지갑 블랙 / 가격:32400 / 제품개수:1
5891 브랜드:블랭크룸 / 제품명:피그먼트 다이 티 셔츠_챠콜 / 가격:32400 / 제품개수:1
5892 브랜드:블랭크룸 / 제품명:3/13 배송 피케 스트라이프 롱슬리브_블랙 / 가격:32400 / 제품개수:1
5893 브랜드:무신사 스탠다드 / 제품명:쉐르파 플리스 집업 재킷 [베이지] / 가격:32400 / 제품개수:1
5894 브랜드:필루미네이트 / 제품명:유니섹스 스탠다드 로고 스웨트 셔츠-옐로우 / 가격:3

6494 브랜드:어반스터프 / 제품명:아트워크 스웨트셔츠 (그레이) / 가격:28000 / 제품개수:1
6495 브랜드:트래블 / 제품명:트래블러 저지 팬츠 블랙 / 가격:28000 / 제품개수:1
6496 브랜드:스윗피쉬 / 제품명:SD High - R2D2 Blue / 가격:28000 / 제품개수:1
6497 브랜드:스컬피그 / 제품명:베이직 고리 레깅스 코코넛베이지 / 가격:28000 / 제품개수:1
6498 브랜드:스컬피그 / 제품명:베이직 고리 레깅스 블랙 / 가격:28000 / 제품개수:1
6499 브랜드:스칼렛또 / 제품명:894 [2SET] 성모마리아X엘리자베스 레이어드 목걸이 / 가격:28000 / 제품개수:1
6500 브랜드:스컬피그 / 제품명:베이직 고리 레깅스 차콜 / 가격:28000 / 제품개수:1
6501 브랜드:세인트페인 / 제품명:SP 리그 로고 티-브라운 / 가격:28000 / 제품개수:1
6502 브랜드:세인트페인 / 제품명:SP 파라다이스 로고 티-블랙 / 가격:28000 / 제품개수:1
6503 브랜드:카시오 / 제품명:A158WA-1DF (A158WA-1) 디지털 공용 메탈시계 32X25mm / 가격:28000 / 제품개수:1
6504 브랜드:트래블 / 제품명:티턴 로케이션 피그먼트 스웨트셔츠 블랙 차콜 / 가격:28000 / 제품개수:1
6505 브랜드:로얄라이프 / 제품명:RLL201  멀티 로고 롱 슬리브 - 블랙 / 가격:28000 / 제품개수:1
6506 브랜드:언더유스 / 제품명:UY 보아 후리스 하이넥 집업 01(브라운) / 가격:28000 / 제품개수:1
6507 브랜드:슬로우애시드 / 제품명:[UNISEX] 레드탭 캡 (블랙) / 가격:28000 / 제품개수:1
6508 브랜드:리플레이컨테이너 / 제품명:RC pocket jogger pants (black) / 가격:28000 / 제품개수:1
6509 브랜드:뉴해튼 / 제품명:워싱 버킷햇 (charcoal) / 가격:28000 / 제품개수:1
6510 브랜

7093 브랜드:어반플레이어스 / 제품명:바서티 후드 티셔츠 (네이비) / 가격:19900 / 제품개수:1
7094 브랜드:어반플레이어스 / 제품명:바서티 후드 티셔츠 (그린) / 가격:19900 / 제품개수:1
7095 브랜드:레이어 유니온 / 제품명:PATCH BOX LOGO SWEAT SHIRTS CHARCOAL / 가격:19900 / 제품개수:1
7096 브랜드:언더유스 / 제품명:유스 인사이드 스탠다드 후드 02(퍼플) / 가격:19900 / 제품개수:1
7097 브랜드:브렌슨 / 제품명:루즈핏 조거 트레이닝 스웨트 팬츠 3컬러 / 가격:19900 / 제품개수:1
7098 브랜드:슈펜 / 제품명:[9차 재입고]슈탠다드 스니커즈 AFHC19W11 / 가격:19900 / 제품개수:1
7099 브랜드:스파오 / 제품명:[1157537]토이스토리 카라 스웨트 셔츠_SPMB949G03 / 가격:19900 / 제품개수:1
7100 브랜드:클라비스 / 제품명:오픈 카라 셔츠(2colors) CVYW94902Q / 가격:19900 / 제품개수:1
7101 브랜드:무신사 스탠다드 / 제품명:라운드넥 라이트 다운 베스트 [블랙] / 가격:19900 / 제품개수:1
7102 브랜드:무신사 스탠다드 / 제품명:핀스트라이프 테이퍼드 밴딩 크롭 슬랙스 [블랙] / 가격:19900 / 제품개수:1
7103 브랜드:에이지 그레이 / 제품명:AI03FBK (Black) SMOOTH 백팩 블랙 / 가격:19900 / 제품개수:1
7104 브랜드:앤드지 / 제품명:브이넥 컬러 포인트 맨투맨 (NV)_BZY1TR1102NV / 가격:19900 / 제품개수:1
7105 브랜드:언더유스 / 제품명:아치 로고 스탠다드 후드 03(다크 그린) / 가격:19900 / 제품개수:1
7106 브랜드:후아유 / 제품명:처짐 없는 9단 수납 백팩[핑크] / WHAK94801A / 가격:19900 / 제품개수:1
7107 브랜드:엠블러 / 제품명:TEDDY 오버핏 자수 반팔티셔츠 AS627 (black)

7419 브랜드:일오공칠 / 제품명:[10PACK] 1507 모노 트라우져 하프 삭스 _ 화이트/블랙 / 가격:13900 / 제품개수:1
7420 브랜드:무신사 스탠다드 / 제품명:베이식 코튼 버킷 햇 [블랙] / 가격:13900 / 제품개수:1
7421 브랜드:무신사 스탠다드 / 제품명:롱 슬리브 크루 넥 티셔츠 [블랙] / 가격:13900 / 제품개수:1
7422 브랜드:무신사 스탠다드 / 제품명:롱 슬리브 크루 넥 티셔츠 [화이트] / 가격:13900 / 제품개수:1
7423 브랜드:더룸 / 제품명:TheLoom logo t-shirt_white / 가격:13900 / 제품개수:1
7424 브랜드:엘라고 / 제품명:에어팟프로 클리어 행 케이스 (젤리) / 가격:13900 / 제품개수:1
7425 브랜드:나이키 / 제품명:스우시 헤어밴드 (화이트) / 가격:13900 / 제품개수:1
7426 브랜드:무신사 스탠다드 / 제품명:롱 슬리브 크루 넥 티셔츠 [크림] / 가격:13900 / 제품개수:1
7427 브랜드:무신사 스탠다드 / 제품명:롱 슬리브 크루 넥 티셔츠 [스모크 핑크] / 가격:13900 / 제품개수:1
7428 브랜드:무신사 스탠다드 / 제품명:롱 슬리브 크루 넥 티셔츠 [피콕 블루] / 가격:13900 / 제품개수:1
7429 브랜드:무신사 스탠다드 / 제품명:롱 슬리브 크루 넥 티셔츠 [멜란지 그레이] / 가격:13900 / 제품개수:1
7430 브랜드:엘라고 / 제품명:에어팟 행 키링 케이스 (1/2세대) / 가격:13900 / 제품개수:1
7431 브랜드:일오공칠 / 제품명:[10PACK] 1507 모노 뱀부 히든 삭스 _ 아이보리/블랙 (페이크 삭스) / 가격:13900 / 제품개수:1
7432 브랜드:무신사 스탠다드 / 제품명:롱 슬리브 크루 넥 티셔츠 [라벤더] / 가격:13900 / 제품개수:1
7433 브랜드:트래블 / 제품명:펭귄 스웨트셔츠 아이보리 / 가격:13900 / 제품개수:1
7434 브랜드:더룸 / 제품명:TheLo

In [28]:
# 모든 제품, 가격, 제품갯수 내림차순으로 출력해서 가장 싼 제품이 무엇인지 알아보자.

import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='watson1259@', db='musinsa', charset='utf8')
cursor = db.cursor()

sql = """SELECT Brand, Product_title, price, COUNT(*) FROM musinsa_best_products GROUP BY Product_title ORDER BY price;"""
cursor.execute(sql)
results = cursor.fetchall()
for index, result in enumerate(results):
    print(index+1, f"브랜드:{result[0]} / 제품명:{result[1]} / 가격:{result[2]} / 제품개수:{result[3]}") 

1 브랜드:앤커버 / 제품명:Smile ball bruin-pink(keyring) / 가격:3000 / 제품개수:1
2 브랜드:엘엠씨 / 제품명:LMC SHOW SOCKS white / 가격:3000 / 제품개수:1
3 브랜드:스파오 / 제품명:(레드벨벳 굿럭트롤) 틴거울_SPFTAA1A21 / 가격:3500 / 제품개수:1
4 브랜드:스컬피그 / 제품명:라인삭스 블랙 / 가격:3500 / 제품개수:1
5 브랜드:스파오 / 제품명:(레드벨벳 굿럭트롤) 스티커팩_SPFTAA1A30 / 가격:3900 / 제품개수:1
6 브랜드:아디다스 / 제품명:Q1 GR 양말 1팩 - 화이트:라오라 / DM7262 / 가격:3900 / 제품개수:1
7 브랜드:네이키드니스 / 제품명:LOGO KEY HOLDER(12COLOR) / 가격:4900 / 제품개수:1
8 브랜드:어커버 / 제품명:베이직 터틀넥 니트 / 가격:4900 / 제품개수:1
9 브랜드:글로리아덴 / 제품명:데일리  볼 귀걸이 / GTE-035 / 가격:5400 / 제품개수:1
10 브랜드:라템 / 제품명:[에브리데이] 샤이닝 스톤 귀걸이_귀걸이(AGIS9V21BBSW) / 가격:5900 / 제품개수:1
11 브랜드:스파오 / 제품명:(레드벨벳 굿럭트롤) 스마트톡_덤덤_SPFTAA1A19 / 가격:5900 / 제품개수:1
12 브랜드:커버낫 / 제품명:AUTHENTIC LOGO GRIPTOK BLACK / 가격:6320 / 제품개수:1
13 브랜드:커버낫 / 제품명:ICON LOGO POUCH BAG BLACK / 가격:6450 / 제품개수:1
14 브랜드:모카썸 / 제품명:미니멀 루즈 스웻셔츠 / 가격:6490 / 제품개수:1
15 브랜드:스칼렛또블랙 / 제품명:B023 [써지컬스틸] 베이직 커브 반지 / 가격:6500 / 제품개수:1
16 브랜드:에프씨엠엠 / 제품명:올라운드 리스트 밴드 / 가격:6500 / 제품개수:1
17 브랜드:트리플에이 / 제품명:트리플에이 무지티 반팔티셔츠 1301 / 가격:6600

669 브랜드:크리틱 / 제품명:EASY SPORTS SHORTS(BLUE)_CTOGUSP03UB2 / 가격:20650 / 제품개수:1
670 브랜드:크럼프 / 제품명:파이핑 트랙 팬츠 (CP0104) / 가격:20700 / 제품개수:1
671 브랜드:위글위글 / 제품명:자수케이스 / 가격:20700 / 제품개수:1
672 브랜드:필루미네이트 / 제품명:유니섹스 베이직 하프 폴라넥-블랙 / 가격:20700 / 제품개수:1
673 브랜드:리얼컨텐츠 / 제품명:[리얼컨텐츠] 하프라운드 S 논인그레이브 925 silver 반지(각인불가능) / 가격:20700 / 제품개수:1
674 브랜드:위글위글 / 제품명:Embroidery Case 아이폰 자수케이스 시즌3 / 가격:20700 / 제품개수:1
675 브랜드:크리틱 / 제품명:WORKING KILLS HOODIE(BLACK)_CTOGPHD06UC6 / 가격:20700 / 제품개수:1
676 브랜드:척 / 제품명:19FW CHK 로고 프린팅 스웨트셔츠 (에메랄드 그린) / 가격:20700 / 제품개수:1
677 브랜드:챔피온 / 제품명:[S0888NA] 파워블랜드 크루 - 네이비 / 가격:20900 / 제품개수:1
678 브랜드:아케시 / 제품명:normal stripe cardigan-white / 가격:20900 / 제품개수:1
679 브랜드:챔피온 / 제품명:[S700DG] 더블 드라이 에코 플리스 후드 - 다크 그린 / 가격:20900 / 제품개수:1
680 브랜드:엠블러 / 제품명:3/10 배송 Hey-Buddy 오버핏 반팔 티셔츠 AS721 (white) / 가격:20900 / 제품개수:1
681 브랜드:무신사 스탠다드 / 제품명:퍼티그 팬츠 [베이지] / 가격:20900 / 제품개수:1
682 브랜드:무신사 스탠다드 / 제품명:코듀로이 오버사이즈 셔츠 재킷 [더스티 베이지] / 가격:20900 / 제품개수:1
683 브랜드:엠블러 / 제품명:3/10 배송 UFO Bear 오버핏 반팔 티셔츠 AS727 (

1169 브랜드:유베스코 / 제품명:프레즌트카드 맨투맨 라벤더 BJHD8078 / 가격:27900 / 제품개수:1
1170 브랜드:유베스코 / 제품명:크루넥미니포켓 맨투맨 그레이 BJMT8079 / 가격:27900 / 제품개수:1
1171 브랜드:유베스코 / 제품명:크루넥미니포켓 맨투맨 아이보리 BJMT8079 / 가격:27900 / 제품개수:1
1172 브랜드:유베스코 / 제품명:프레즌트카드 맨투맨 스카이블루 BJHD8078 / 가격:27900 / 제품개수:1
1173 브랜드:칼하트 / 제품명:오데사 볼캡 아스팔트 / 100289-APH / 가격:27900 / 제품개수:1
1174 브랜드:나이키 / 제품명:스우시 에센셜 미드컷 삭스 (3팩) / 가격:27900 / 제품개수:1
1175 브랜드:아디다스 / 제품명:에센셜 티 - 화이트 / FM9966 / 가격:27900 / 제품개수:1
1176 브랜드:시그니처 / 제품명:[20SS]스트링 투웨이 트레이닝 팬츠[블랙] / 가격:27900 / 제품개수:1
1177 브랜드:유니스디자인 / 제품명:릴렉스핏 코듀로이 자켓_블랙 / 가격:27900 / 제품개수:1
1178 브랜드:집시 / 제품명:오버사이즈 쿨링베이스 티셔츠 -아이보리- / 가격:27900 / 제품개수:1
1179 브랜드:무신사 스탠다드 / 제품명:오픈 칼라 롱 슬리브 셔츠 [네이비] / 가격:27900 / 제품개수:1
1180 브랜드:무신사 스탠다드 / 제품명:오픈 칼라 롱 슬리브 셔츠 [스모크 핑크] / 가격:27900 / 제품개수:1
1181 브랜드:무신사 스탠다드 / 제품명:오픈 칼라 롱 슬리브 셔츠 [화이트] / 가격:27900 / 제품개수:1
1182 브랜드:무신사 스탠다드 / 제품명:오픈 칼라 롱 슬리브 셔츠 [다크 그레이] / 가격:27900 / 제품개수:1
1183 브랜드:무신사 스탠다드 / 제품명:와이드 스프레드 칼라 드레스 셔츠 [스카이 블루] / 가격:27900 / 제품개수:1
1184 브랜드:무신사 스탠다드 / 제품명:오픈 칼라 롱 

1698 브랜드:무신사 스탠다드 / 제품명:세미 와이드 히든 밴딩 슬랙스 [라이트 브라운] / 가격:31900 / 제품개수:1
1699 브랜드:무신사 스탠다드 / 제품명:세미 와이드 히든 밴딩 슬랙스 [핑크 베이지] / 가격:31900 / 제품개수:1
1700 브랜드:무신사 스탠다드 / 제품명:세미 와이드 히든 밴딩 슬랙스 [라이트 그레이] / 가격:31900 / 제품개수:1
1701 브랜드:무신사 스탠다드 / 제품명:세미 와이드 히든 밴딩 슬랙스 [다크 그레이] / 가격:31900 / 제품개수:1
1702 브랜드:무신사 스탠다드 / 제품명:세미 와이드 슬릿 히든 밴딩 슬랙스 [차콜 그레이] / 가격:31900 / 제품개수:1
1703 브랜드:무신사 스탠다드 / 제품명:테이퍼드 히든 밴딩 크롭 슬랙스 [차콜 그레이] / 가격:31900 / 제품개수:1
1704 브랜드:무신사 스탠다드 / 제품명:와이드 히든 밴딩 크롭 슬랙스 [블랙] / 가격:31900 / 제품개수:1
1705 브랜드:무신사 스탠다드 / 제품명:라이트웨이트 세미 와이드 히든 밴딩 슬랙스 [블랙] / 가격:31900 / 제품개수:1
1706 브랜드:제멋 / 제품명:3/17 배송 엔트 투웨이 후드집업 그레이 KJHD2272 / 가격:31900 / 제품개수:2
1707 브랜드:무신사 스탠다드 / 제품명:와이드 히든 밴딩 슬랙스 [크림] / 가격:31900 / 제품개수:1
1708 브랜드:무신사 스탠다드 / 제품명:와이드 히든 밴딩 슬랙스 [다크 그레이] / 가격:31900 / 제품개수:1
1709 브랜드:무신사 스탠다드 / 제품명:테이퍼드 히든 밴딩 크롭 슬랙스 [화이트] / 가격:31900 / 제품개수:1
1710 브랜드:무신사 스탠다드 / 제품명:리얼 와이드 히든 밴딩 슬랙스 [차콜 그레이] / 가격:31900 / 제품개수:1
1711 브랜드:엠블러 / 제품명:TEDDY 오버핏  스웨트셔츠 AMM803(purple) / 가격:31900 / 제품개수:1
1712 브랜드:엠블러 / 제품명:TEDDY 

2169 브랜드:라퍼지스토어 / 제품명:[LAFITS] 링클프리 세미 와이드 밴딩 슬랙스_Cream Beige / 가격:36000 / 제품개수:1
2170 브랜드:라퍼지스토어 / 제품명:[LAFITS] 세미 와이드 코튼 밴딩 슬랙스_Cream / 가격:36000 / 제품개수:1
2171 브랜드:라퍼지스토어 / 제품명:[LAFITS] 스트레이트 밴딩 크롭 진 _Natural Cream / 가격:36000 / 제품개수:1
2172 브랜드:라퍼지스토어 / 제품명:[LAFITS] 세미 와이드 코튼 밴딩 슬랙스_Black / 가격:36000 / 제품개수:1
2173 브랜드:라퍼지스토어 / 제품명:[LAFITS] 스트레이트 밴딩 크롭 워싱 진_Blue / 가격:36000 / 제품개수:1
2174 브랜드:커스텀어클락 / 제품명:소프트 베이직 가디건 빈티지라이트블루 / 가격:36000 / 제품개수:1
2175 브랜드:더스토리 / 제품명:레귤러핏 크롭 팬츠 (블랙) / 가격:36000 / 제품개수:1
2176 브랜드:네스티팬시클럽 / 제품명:[NF] FANCY LAB POCKET MINI SKIRT (GRAY) (19FW-F702) / 가격:36000 / 제품개수:1
2177 브랜드:커스텀어클락 / 제품명:소프트 베이직 가디건 빈티지브라운 / 가격:36000 / 제품개수:1
2178 브랜드:커스텀어클락 / 제품명:소프트 베이직 가디건 오트밀 / 가격:36000 / 제품개수:1
2179 브랜드:커버낫 / 제품명:COTTON USA WAPPEN ARCH LOGO CREWNECK GRAY / 가격:36000 / 제품개수:1
2180 브랜드:라퍼지스토어 / 제품명:3/20 배송 오디너리 카고 조거 슬랙스_Neutral Beige / 가격:36000 / 제품개수:1
2181 브랜드:브라바도 / 제품명:GNR APPETITE (BRENT1622) / 가격:36000 / 제품개수:1
2182 브랜드:커스텀어클락 / 제품명:소프트 베이직 가디건 베이지 / 가격:36000 / 제품개수:1
2

...

4786 브랜드:슈펜 / 제품명:[체스트벨트 추가] 네스티킥 콜라보 멀티백팩 블랙 TPBK19W82 / 가격:59900 / 제품개수:1
4787 브랜드:엘무드 / 제품명:화란 세미오버 니트 터콰이즈 / 가격:59900 / 제품개수:1
4788 브랜드:엘무드 / 제품명:3/9 배송 화란 세미오버 니트 블루 스톤 / 가격:59900 / 제품개수:1
4789 브랜드:엘무드 / 제품명:화란 세미오버 니트 블랙 / 가격:59900 / 제품개수:1
4790 브랜드:무신사 스탠다드 / 제품명:릴렉스드 데님 트러커 재킷 [워시드 블랙] / 가격:59900 / 제품개수:1
4791 브랜드:칼하트 / 제품명:미드웨이트 로고 후드 블랙 / K288-001 / 가격:59900 / 제품개수:1
4792 브랜드:엘무드 / 제품명:화란 세미오버 니트 제주 / 가격:59900 / 제품개수:1
4793 브랜드:엘무드 / 제품명:화란 세미오버 니트 모카 베이지 / 가격:59900 / 제품개수:1
4794 브랜드:엘무드 / 제품명:화란 세미오버 니트 그레이 / 가격:59900 / 제품개수:1
4795 브랜드:파르티멘토 / 제품명:3/24 배송 후디 필드 자켓 본 화이트 / 가격:59900 / 제품개수:1
4796 브랜드:엘무드 / 제품명:화란 세미오버 니트 민트그레이 / 가격:59900 / 제품개수:1
4797 브랜드:엘무드 / 제품명:화란 세미오버 니트 크림 브륄레 / 가격:59900 / 제품개수:1
4798 브랜드:파르티멘토 / 제품명:보아 플리스 집업 자켓 아이보리 / 가격:59900 / 제품개수:1
4799 브랜드:엘무드 / 제품명:3/9 배송 화란 세미오버 니트 라벤더 / 가격:59900 / 제품개수:1
4800 브랜드:유니온가먼츠 / 제품명:와이드 스트레이트 데님 팬츠 02 / 가격:59900 / 제품개수:1
4801 브랜드:아디다스 / 제품명:락업 후디 - 블랙 / ED6124 / 가격:59900 / 제품개수:1
4802 브랜드:리복 / 제품명:스냅팬츠 - 네이비:화이트 / FP8

5373 브랜드:잇터 / 제품명:빈티지 브라운 가먼트 워싱 스웻셔츠 / 가격:69000 / 제품개수:1
5374 브랜드:마크 곤잘레스 / 제품명:M/G P-PARODY ANGEL CREWNECK INDIGO BLUE / 가격:69000 / 제품개수:1
5375 브랜드:반스 / 제품명:올드스쿨 - 네이비 / VN000D3HNVY1 / 가격:69000 / 제품개수:1
5376 브랜드:반스 / 제품명:올드스쿨 - (Primary Check) 블랙:화이트 / VN0A38G1P0S1 / 가격:69000 / 제품개수:1
5377 브랜드:카파 / 제품명:222반다 HOLIC 운동화 베이지 KKWK311N1 / 가격:69000 / 제품개수:1
5378 브랜드:오찌 / 제품명:LOW PU_IRIS BLUE / 가격:69000 / 제품개수:1
5379 브랜드:휠라 / 제품명:트레이서 TR(1RM01279147) / 가격:69000 / 제품개수:1
5380 브랜드:오드스튜디오 / 제품명:3/31 배송 [사은품 3종 증정] 오디디 스카치 매쉬 백팩-BLACK / 가격:69000 / 제품개수:1
5381 브랜드:커버낫 / 제품명:AUTHENTIC LOGO CREWNECK NAVY / 가격:69000 / 제품개수:1
5382 브랜드:헤드 / 제품명:남녀공용 스크래퍼 벨크로 레트로 어글리슈즈_KF4DX19827BKX / 가격:69000 / 제품개수:1
5383 브랜드:커버낫 / 제품명:SMALL AUTHENTIC LOGO CREWNECK BLACK / 가격:69000 / 제품개수:1
5384 브랜드:와일드 브릭스 / 제품명:MALLARD CARD CASE (brown) / 가격:69000 / 제품개수:1
5385 브랜드:디프리크 / 제품명:06 Layered T-Shirt - Black / 가격:69000 / 제품개수:1
5386 브랜드:에이지 / 제품명:타탄 체크 컷 AGE TARTAN CHECK CUT (AGFT-CR-CT-CKG-011) / 가격:69000 / 제품개수:1
5387

5917 브랜드:나이키 / 제품명:스우시 트레이닝세트 블랙+레드 / 가격:79000 / 제품개수:1
5918 브랜드:이십삼점육오 / 제품명:3/30 배송 V2 BLACK / 가격:79000 / 제품개수:1
5919 브랜드:프리즘웍스 / 제품명:OG PIGMENT DYEING HOODY _ BLUE / 가격:79000 / 제품개수:1
5920 브랜드:프리즘웍스 / 제품명:HEAVYWEIGHT STRING ZIP UP HOODY _ GRAY / 가격:79000 / 제품개수:1
5921 브랜드:코이 / 제품명:WIDE BALLOON PANTS BLACK / 가격:79000 / 제품개수:1
5922 브랜드:그루브라임 / 제품명:REFLECTIVE CHUNKY UGLY SHOES (WHITE) [GSN001I53WH] / 가격:79000 / 제품개수:1
5923 브랜드:매스노운 / 제품명:바이오로지컬 심볼 오버사이즈 후드티 MSZHD001-DG / 가격:79000 / 제품개수:1
5924 브랜드:휠라 / 제품명:베이직 솔리드 아노락(FS2JKC1104XLPU) / 가격:79000 / 제품개수:1
5925 브랜드:가먼트레이블 / 제품명:3/13 배송 [2차 리오더]GL Minimal Holster Bag / 가격:79000 / 제품개수:1
5926 브랜드:엠엘비 / 제품명:모노그램 웨이스트백 NY (IVORY) / 가격:79000 / 제품개수:1
5927 브랜드:반스 / 제품명:클래식 슬립온 98 DX - (애너하임 팩토리)  - 옐로우 네온:체커보드 / VN0A3JEXV9O1 / 가격:79000 / 제품개수:1
5928 브랜드:나이키 / 제품명:스우시 더플백 / 가격:79000 / 제품개수:1
5929 브랜드:콰이어티스트 / 제품명:8골 코듀로이 믹스 셔츠 자켓 (ivory) / 가격:79000 / 제품개수:1
5930 브랜드:이십삼점육오 / 제품명:3/30 배송 V2 Shoes / 가격:79000 / 제품개수:1
5931 브랜드:모프 / 제품명:사이드 라인 팬츠 그레

6619 브랜드:리올그 / 제품명:SENTIMENTAL SKIRT LIGHT VIOLET / 가격:104500 / 제품개수:1
6620 브랜드:앤더슨벨 / 제품명:유니섹스 인사이드 아웃 콘트사리스 패널 후드 atb345u(BLACK/LIME) / 가격:105000 / 제품개수:1
6621 브랜드:로맨틱크라운 / 제품명:BUTTON FLEECE JUMPER_OATMEAL / 가격:105000 / 제품개수:1
6622 브랜드:레미 / 제품명:DOTTED MESH POLKA TUTU SKIRT (CHARCOAL) / 가격:105000 / 제품개수:1
6623 브랜드:컨버스 / 제품명:척 70 트위스티드 프렙 블랙 166855C / 가격:105000 / 제품개수:1
6624 브랜드:컨버스 / 제품명:척 70 시즈널 컬러 스웨이드 백앨리브릭 166493C / 가격:105000 / 제품개수:1
6625 브랜드:앤더슨벨 / 제품명:유니섹스 인사이드 아웃 콘트사리스 패널 후드  atb345u(S.BLUE/BEIGE) / 가격:105000 / 제품개수:1
6626 브랜드:미니멀프로젝트 / 제품명:와이드 벌룬 블루종 자켓 MJK105 [KHAKI] / 가격:105000 / 제품개수:1
6627 브랜드:미니멀프로젝트 / 제품명:와이드 벌룬 블루종 자켓 MJK105 [BLACK] / 가격:105000 / 제품개수:1
6628 브랜드:아디다스 / 제품명:오즈위고 - 트리플그레이 / EG0547 / 가격:105900 / 제품개수:1
6629 브랜드:모드나인 / 제품명:Night Hawk - MOD4 crop / 가격:106000 / 제품개수:1
6630 브랜드:모드나인 / 제품명:Moonlight Blue - MOD1 / 가격:106000 / 제품개수:1
6631 브랜드:아조바이아조 오리지널 라벨 / 제품명:3/9 배송 Oversized Slogan Wool Knit Sweater [Lilac] / 가격:106200 / 제품개수:1
6632 브랜드:모드나인 / 제품명:Uncanny B

7166 브랜드:제로 / 제품명:M-51 Fishtail Parka [Khaki] / 가격:149000 / 제품개수:1
7167 브랜드:클라시코 / 제품명:Fanone Sneakers_Ivory / 가격:149000 / 제품개수:1
7168 브랜드:아식스 / 제품명:젤-카야노 5 OG - 화이트:화이트 / 1191A149-100 / 가격:149000 / 제품개수:1
7169 브랜드:지프 / 제품명:탱크 체로키 (GK0GHU101BK) / 가격:149000 / 제품개수:1
7170 브랜드:리바이스 / 제품명:남성 LEJ 트러커 자켓_67778-0001 / 가격:149000 / 제품개수:1
7171 브랜드:드로우핏 / 제품명:오버사이즈 트렌치 코트 [BEIGE] / 가격:149000 / 제품개수:1
7172 브랜드:드로우핏 / 제품명:3/19 배송 오버사이즈 트렌치 코트 [BLACK] / 가격:149000 / 제품개수:1
7173 브랜드:드로우핏 / 제품명:3/20 배송 오버사이즈 트렌치 코트 [L.BEIGE] / 가격:149000 / 제품개수:1
7174 브랜드:드로우핏 / 제품명:오버사이즈 M-51 코트 자켓 [KHAKI] / 가격:149000 / 제품개수:1
7175 브랜드:나이키 / 제품명:남성  테크 팩 후드 풀집 (BV4490-010) / 가격:149000 / 제품개수:1
7176 브랜드:셉템버나인 / 제품명:Model. K2F2WH Man Sneakers 스니커즈 SEPTEMBER NINE(셉템버나인) / 가격:149000 / 제품개수:1
7177 브랜드:플레어업 / 제품명:오버사이즈 트렌치코트 (FL-012) / 가격:149000 / 제품개수:1
7178 브랜드:드로우핏 / 제품명:오버사이즈 트렌치 코트 [L.GREY] / 가격:149000 / 제품개수:1
7179 브랜드:널디 / 제품명:코듀로이 벨벳 다운 자켓 블랙 / 가격:149000 / 제품개수:1
7180 브랜드:드로우핏 / 제품명:오버사이즈 트렌치 코트 [CHARCOAL] / 가격:1490

### 1.가장 비싼 제품 = 브랜드:몽클레어 / 제품명:테렌스 1038900 01AJQ 002 남자 스니커즈 / 가격:775000 / 제품개수:1
### 2-1.가장 싼 제품 = 브랜드:앤커버 / 제품명:Smile ball bruin-pink(keyring) / 가격:3000 / 제품개수:1 
### 2-2.가장 싼 제품 = 브랜드:엘엠씨 / 제품명:LMC SHOW SOCKS white / 가격:3000 / 제품개수:1

### 위에서 '무신사 스토어의 오류' 라고 칭했던 가장 비싼 제품, 싼 제품 쿼리를 날렸을 때 추출된 제품들과 실제 가장 비싸고 싼 가격의 제품 가격이 똑같다. 이는 아마도 SQL서버 내에서 똑같은 가격의 제품 여러 개 중 '무신사 스탠다드 - 베이식 블레이저' 제품을 추출한 것 같다. 어찌되었든 홈페이지 측의 가격 책정 오류는 일부 맞는 듯 하다

# DB에 데이터 넣기  TEST

In [98]:
data_dict = {'ranking': '1', 'brand': '무신사 스탠다드', 'title': '베이식 블레이저 [블랙]', 'price': '69900'}

In [102]:
sql = """INSERT INTO musinsa_best_products values("""+data_dict['ranking']+""",'"""+data_dict['brand']+"""',
'"""+data_dict['title']+"""',"""+data_dict['price']+""")"""
print(sql.replace('\n',''))

INSERT INTO musinsa_best_products values(1,'무신사 스탠다드','베이식 블레이저 [블랙]',69900)


##  llll 크롤링 Test llll

In [78]:
import requests
from bs4 import BeautifulSoup

result = requests.get("https://store.musinsa.com/app/contents/bestranking/?d_cat_cd=&u_cat_cd=&range=nw&price=&ex_soldout=&sale_goods=&new_product_yn=&list_kind=&page=2")
soup = BeautifulSoup(result.content, 'html.parser')

products = soup.select("div.list-box.box li")
for product in products:
    result = product.select_one("div.article_info")
    if result != None:
        if result.select_one('p.item_title:nth-of-type(0) > span'):
            brand = result.select_one('p.item_title:nth-of-type(1)').get_text()
        else:
            brand = result.select_one('p.item_title > a').get_text()
        title = result.select_one('p.list_info > a').get_text().strip()
        price = result.select_one('p.price')
        origin = price.select_one('del')
        if origin:
            _ = origin.extract()
            price = price.get_text().strip().replace('원','').replace(',','')
        else:
            price = price.get_text().strip().replace('원','').replace(',','')
        
        
    ranking = product.find('p', {'class':'n-label'})
    if ranking != None:
        increase = ranking.find('span')
        if increase:
            _ = increase.extract()
            ranking = ranking.get_text().strip()
        else:
            ranking = ranking.get_text().strip()
        
        print(ranking, brand, title, price)

            
        
    
        
    


81위 알파 인더스트리 [육성재]MA-1 레귤러핏 - 세이지 그린 / MJM21000C1-SGR 215000 2174
82위 엘무드 3/5 배송 화란 세미오버 가디건 블랙 63800 1037
83위 알파 인더스트리 [빈지노]MA-1 레귤러핏 - 건메탈 / MJM21000C1-GMT 215000 1535
84위 내셔널지오그래픽 N201MJP920 로우머스 플라이트 블루종 점퍼 CARBON BLACK 219000 95
85위 아디다스 3-스트라이프 트레이닝 팬츠 - 와인 / CM1410 56900 6033
86위 에스피오나지 NAVAL Academy Sweat Zip Hoodie Grey 89000 339
87위 슈펜 [4차재입고][최겨울x슈펜]이태리 비브람 시리즈 더비슈즈 BYSC19W04 99000 3800
88위 써드위브 [모델 주우재]ULTIMATE BACKPACK / BLACK-SCOTCH 70900 5748
89위 에스피오나지 3/6 배송 NAVAL Academy Sweat Hoodie Grey 71000 946
90위 커버낫 LAYOUT LOGO HOODIE ZIP-UP GRAY 55300 1549
91위 토피 3/20 배송 와이드 데님 팬츠 (LIGHT BLUE) 44100 21556
92위 아디다스 에센셜 3S 윈드 팬츠 - 블랙:화이트 / DQ3100 51900 19112
93위 엘무드 화란 세미오버 니트 애쉬 블루 59900 1463
94위 비바스튜디오 [WayV 샤오쥔]LOCATION CREWNECK JS [BLACK] 52200 13664
95위 디스커버리 익스페디션 [배우 김보라]버킷 디워커 V2 (BEIGE) 139000 1190
96위 무신사 스탠다드 라이트웨이트 테이퍼드 밴딩 크롭 슬랙스 [블랙] 23900 10820
97위 86로드 2724 Washing denim jacket (L.blue) 69000 1480
98위 커버낫 [박수지]커버낫X피너츠 70th C 로고 크루넥 아이보리 69000 482
99위 도프제이슨 램스킨 100